# 30dim Branin Benchmark with SAASBO
This is a port from https://github.com/pytorch/botorch/blob/main/tutorials/saasbo.ipynb
## Imports

In [13]:
from bofire.benchmarks.single import Branin30
from bofire.runners.api import run
from bofire.data_models.strategies.api import SoboStrategy, RandomStrategy
from bofire.data_models.surrogates.api import SaasSingleTaskGPSurrogate, BotorchSurrogates
from bofire.data_models.acquisition_functions.api import qLogEI
import bofire.strategies.api as strategies
from bofire.data_models.api import Domain
import pandas as pd
import os

SMOKE_TEST = os.environ.get("SMOKE_TEST")

N_ITERATIONS = 10 if not SMOKE_TEST else 1
BATCH_SIZE = 5 if not SMOKE_TEST else 1
WARMUP_STEPS = 256 if not SMOKE_TEST else 32
NUM_SAMPLES = 128 if not SMOKE_TEST else 16
THINNING = 16

## Random Optimization

In [5]:
def sample(domain):
    datamodel = RandomStrategy(domain=domain)
    sampler = strategies.map(data_model=datamodel)
    sampled = sampler.ask(10)
    return sampled

def best(domain: Domain, experiments: pd.DataFrame) -> float:
    return experiments.y.min()

random_results = run(
    Branin30(),
    strategy_factory=lambda domain: strategies.map(RandomStrategy(domain=domain)),
    n_iterations=N_ITERATIONS,
    metric=best,
    initial_sampler=sample,
    n_candidates_per_proposal=5,
    n_runs=1,
    n_procs=1,
)

run 00 with current best 6.406: 100%|██████████| 10/10 [00:00<00:00, 33.56it/s]


## SAASBO Optimization

In [9]:
benchmark = Branin30()

def strategy_factory(domain: Domain):
    data_model = SoboStrategy(
        domain=domain,
        acquisition_function=qLogEI(),
        surrogate_specs=BotorchSurrogates(
            surrogates=[
                SaasSingleTaskGPSurrogate(inputs=benchmark.domain.inputs, outputs=benchmark.domain.outputs,
                                          # the following hyperparams do not need to be provided
                                          warmup_steps=WARMUP_STEPS,
                                          num_samples=NUM_SAMPLES, 
                                          thinning=THINNING)]))
    return strategies.map(data_model)


random_results = run(
    Branin30(),
    strategy_factory=strategy_factory,
    n_iterations=N_ITERATIONS,
    metric=best,
    initial_sampler=sample,
    n_candidates_per_proposal=5,
    n_runs=1,
    n_procs=1,
)

run 00 with current best 0.398: 100%|██████████| 10/10 [18:57<00:00, 113.80s/it]
